# Offline Sunflare addition
This Notebook illustrates the method we adopted to add sunflare effect offline.

In [ ]:
import json
import random
import cv2
import numpy as np
import os
import shutil

# Path to enriched json file
with open('synthetic_train_enriched.json', 'r') as f:
  data = json.load(f)

# Set directories
imgs_source_dir="speedplus\\speedplus\\synthetic\\images\\"
imgs_dest_dir='speedplus\\speedplus\\synthetic\\images_processed_sunflare\\'

#Set the number of images to process
imgs_to_process = int(np.round(1/2*len(data)))

#Randomly sample images
whole_index=list(range(len(data)))
sunflare_imgs_index = random.sample(whole_index,imgs_to_process)
data_sunflare_imgs= [data[i] for i in sunflare_imgs_index]

#Copy all images in the dest folder (processed images will be overwritten)
for i in range(len(data)):
  shutil.copy(imgs_source_dir+data[i]['filename'],imgs_dest_dir)

Define a distribution of radii for the SunFlare effect

In [ ]:
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

nr=len(data_sunflare_imgs)
myclip_a=150
myclip_b=500
my_mean=200
my_std=125
a,b=(myclip_a-my_mean)/my_std,(myclip_b-my_mean)/my_std
radii = np.int32(truncnorm.rvs(a,b,loc=my_mean,scale=my_std,size=nr))
plt.hist(radii, bins=25);

Apply SunFlare effect and save processed images.

In [ ]:
from albumentations import RandomSunFlare
# https://github.com/albumentations-team/albumentations

for i in range(len(data_sunflare_imgs)):
  filename=data_sunflare_imgs[i]['filename']
  image_dir=os.path.join(imgs_source_dir,filename)
  image = cv2.imread(image_dir,cv2.IMREAD_GRAYSCALE)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

  xmin = data_sunflare_imgs[i]['bbox_coords'][0]/1920
  ymin = data_sunflare_imgs[i]['bbox_coords'][1]/1200
  xmax = data_sunflare_imgs[i]['bbox_coords'][2]/1920
  ymax = data_sunflare_imgs[i]['bbox_coords'][3]/1200

  transform=RandomSunFlare(angle_lower=0.5,flare_roi=[xmin,ymin,xmax,ymax], num_flare_circles_lower=0, num_flare_circles_upper=1,src_radius=radii[i],always_apply=True)

  # Augment an image
  transformed = transform(image=image)
  transformed_image = transformed["image"]
  transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2GRAY)
  cv2.imwrite(imgs_dest_dir+filename,transformed_image,[int(cv2.IMWRITE_JPEG_QUALITY), 75])
